In [1]:
srt_fr_path = "The Golden Child.DVD_PAL.fre.srt"
srt_en_path = "The Golden Child (1986) 1080p web.en.srt"

In [2]:
import pysrt

srt_fr = pysrt.open(srt_fr_path)
srt_en = pysrt.open(srt_en_path)

In [3]:
srt_en[0].text, srt_fr[2].text

("Don't let him touch you!", 'Qu’il ne vous touche pas.')

In [4]:
srt_fr.shift(ratio=25.00/23.9)

In [5]:
diff_starts = srt_fr[2].start - srt_en[0].start
print(diff_starts)
#srt_fr.shift(seconds=diff_starts.seconds, milliseconds=diff_starts.milliseconds)

00:00:00,261


In [6]:
[
    (srt_fr[i+2].text, srt_en[i].text, srt_fr[i+2].start - srt_en[i].start)
    for i in range(100)
]

[('Qu’il ne vous touche pas.',
  "Don't let him touch you!",
  SubRipTime(0, 0, 0, 261)),
 ('<i>Tartes aux poils.</i>', '<i>Butt Pie.</i>', SubRipTime(0, 0, 0, 894)),
 ('<i>Gâteaux aux poils,</i>\nc’est la suite d’un bouquin.',
  "There's a sequel to a book written\ncalled <i>Butt Cake.</i>",
  SubRipTime(0, 0, 0, 846)),
 ('Ça marche bien dans les kiosques.\nUn cul plein de gâteau.',
  'Very popular at the news-stands.\nAbout a butt with cake all over it.',
  SubRipTime(0, 0, 0, 842)),
 ('<i>Gros culs</i>', '<i>Chunky Asses?</i>', SubRipTime(0, 0, 0, 950)),
 ('<i>Bienvenue au Journal Backman.\nJe suis Mel Backman.</i>',
  "<i>Welcome to the Mel Backman Journal.</i>\n<i>I'm Mel Backman.</i>",
  SubRipTime(0, 0, 1, 86)),
 ('<i>Je suis en compagnie\nde Chandler Jarrell...</i>',
  '<i>With me today is Chandler Jarrell</i>',
  SubRipTime(0, 0, 1, 123)),
 ('<i>dont l’action est très utile\nà la société.</i>',
  "<i>who's doing something</i>\n<i>very vital for the community.</i>",
  SubRipTim

In [7]:
from googletrans import Translator

translator = Translator()

In [17]:
from itertools import zip_longest
from tqdm import tqdm_notebook as tqdm

def grouper(iterable, n, fillvalue=None):
    "Collect data into fixed-length chunks or blocks"
    # grouper('ABCDEFG', 3, 'x') --> ABC DEF Gxx"
    args = [iter(iterable)] * n
    return zip_longest(*args, fillvalue=fillvalue)

In [18]:
chunk_size = 15

In [ ]:
# don't work ! :'(
for slice_srt_text in tqdm(
    map(lambda g: "".join(g), 
        grouper(map(lambda srt: srt.text, srt_en), chunk_size, fillvalue="")),
    total=len(srt_en) // chunk_size
):
    try:
        t_slice_srt_text = translator.translate(slice_srt_text,
                                                src='en',
                                                dest='fr')
    except Exception as e:
        pass
    else:
        print(f"{t_slice_srt_text.text}")

In [16]:
import urllib.request
from urllib.parse import quote
import sys

typ = sys.getfilesystemencoding()

def translate(querystr, to_l="zh", from_l="en"):
    '''for google tranlate by doom
    '''
    C_agent = {'User-Agent': "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/31.0.165063 Safari/537.36 AppEngine-Google."}
    flag = 'class="t0">'
    tarurl = ("http://translate.google.cn/m?hl=%s&sl=%s&q=%s" % (to_l, from_l, quote(querystr, safe='')))
    request = urllib.request.Request(tarurl, headers=C_agent)
    page = str(urllib.request.urlopen(request).read().decode(typ))
    target = page[page.find(flag) + len(flag):]
    target = target.split("<")[0]
    return target

In [22]:
from urllib.parse import unquote
from xml.sax.saxutils import unescape

for slice_srt_text in tqdm(
    map(lambda g: "".join(g), 
        grouper(map(lambda srt: srt.text, srt_en), chunk_size, fillvalue="")),
    total=len(srt_en) // chunk_size
):
    try:
        t_slice_srt_text = unescape(translate(slice_srt_text, from_l='en', to_l='fr'))
    except Exception as e:
        pass
    else:
        print(f"{t_slice_srt_text}")

Ne le laissez pas vous toucher! <I> Butt Pie. </ I> Il y a une suite à un livre écrit appelé <i> Butt Cake. </ i> Très populaire aux kiosques à journaux. Sur un cul avec un gâteau partout. <I> Chunky Asses? </ I> <i> Bienvenue dans le Mel Backman Journal. </ I> <I> Je suis Mel Backman. </ i> <i> Aujourd&#39;hui, Chandler Jarrell </ i> <i> fait quelque chose </ i> <I> très vital pour la communauté. </ i> <i> Chandler est un découvreur d&#39;enfants perdus. </ I> <I> Ai-je raison? </ i> - <i> Oui, je trouve des enfants disparus. </ I> - <i> C&#39;est très intéressant. </ i> <i> Il y avait un spectacle appelé Finder </ i> <I> of Lost Loves. Avez-vous vu cela? </ I> - <i> Tony Franciosa était la star. </ I> - <i> Ouais ... </ i> - <i> Je cherche cette fille ... </ i> - <i> J&#39;espère que vous n&#39;êtes pas nerveux. </ i> <i> C&#39;est parfaitement normal. C&#39;est </ i> <I> mon cinquième spectacle et je suis toujours nerveux. </ I> <i> En fait, y a-t-il quelqu&#39;un ici </ i> <I> qui 

prendre un verre, et attendez d&#39;autres instructions.Je suis désolé. Pourquoi ne montes-tu pas? pour un moment? Je serai bon.OK.J&#39;ai essayé.Vous devriez toujours donner une femme deux chances de dire non, juste au cas où. C&#39;est &quot;non&quot;, deux fois.Tout droit ... mais les esprits vont être très en colère.Vous savez, je suis l&#39;élu.OK.Vous <i> allez </ i> manger.Obtenez cela, et le reste de vos jouets, Hé, oiseau! Tu viens de voir un peu Hare Krishna Nain dans un arbre, flottant? Ou est-ce moi? Ce doit être Rod Serling.- Ouais? - <i> Recherchez-vous Cheryl Mosely? </ I>
- Ouais. - <I> Je connais les bâtards qui l&#39;ont saisie. </ I> - Avec qui était-elle? - <i> Avez-vous déjà été à Pacoima? </ i> - Ouais. Avec qui était-elle? - <i> Derrière l&#39;autoroute, il y a une maison. </ i> <i> C&#39;est un tapis pour certains motards </ i> <I> appelé les dragons jaunes. </ i> <i> Elle était avec eux la semaine dernière. </ I> <I> Elle n&#39;est pas là maintenant. </ I> Qui

- Et pour &quot;Numsy&quot;? - Numspa. Sardo Numspa.- Oui. - Il est très réel.Vous étiez là et a dit des choses cela semblait assez réel.- Cette partie était un rêve. - Je vérifie juste. Tu ne vas pas me demander? - Est-ce que je rêve maintenant? - Une dame doit-elle demander deux fois? Non! Non ... S&#39;il vous plaît, deux fois! S&#39;il vous plaît, s&#39;il vous plaît. Deux fois! Je ne savais pas si tu voulais crème ou du sucre, alors je l&#39;ai laissé en noir.Je veux que tu ailles au Tibet. Voulez-vous aller? Attendez une seconde. Il fait vraiment froid au Tibet et je n&#39;aime pas le froid. Pourquoi ne pas rester ici? tout le week-end sous les couvertures, regarder la télévision, transpirer, souriant et tout? - Tu ne partiras pas. - Il fait très froid au Tibet.
- Salut, où vas-tu? - Tibet.Mais je suis l&#39;élu. Vous ne pouvez pas obtenir le couteau. Je vais y aller. J&#39;irai au Tibet. Attends ... Mais s&#39;il fait très froid, nous devons revenir ici. OK? Vous n&#39;êtes pas 

J&#39;ai honte de moi. Je devrais être arrêté. Je devrais être purgé, Je devrais être fouetté. Je ne devrais pas marcher parmi les bonnes personnes. - Laisse-moi lui parler. Je suis un porc, un misérable. Je ne mérite pas de vivre comme les autres. Combien de temps pensez-vous vous pouvez garder cette mascarade? Jusqu&#39;à ce que je sois arrêté, ou vous réalisez les règles de preuve dans ce pays. Si je suis arrêté, ils me prendront et me mettre dans une cellule de prison.Ils vont prendre le couteau, parce qu&#39;il est volé, et placez un autocollant «Pièce A» dessus. Le couteau est assis dans une pièce et je suis assis dans ma petite chambre jusqu&#39;à ce que le procès arrive un mois à un an plus tard. Si vous me faites arrêter, rien ne dit. quand tu auras ton couteau. Tu n&#39;as aucune idée de qui je suis, n&#39;est-ce pas? Oui! Vous êtes Sardo Numsy! Je me fiche de qui vous êtes. je m&#39;en préoccupe que vous avez enlevé un enfant, cependant. Je pourrais vous détruire comme ça.
N